# Computation Intelligence for Optimization | Sports League Optimization

`Group AM`
- Eduardo Mendes, 20240850
- Helena Duarte, 20240530
- João Freire, 20240528
- Mariana Sousa, 20240516

<div class="alert alert-block alert-info">

# Table of Contents
    
[1. Import Libraries](#1)<br>

[2. Load data](#2)<br>

<a class="anchor" id="1">

# 1. Import Libraries
    
</a>

In [85]:
import os
import pandas as pd

from copy import deepcopy
from random import random, sample, choice, randint
import copy

<a class="anchor" id="2">

# 2. Load data
    
</a>

In [86]:
#data_dir= os.path.join(os.getcwd(), 'players(in).csv')

df = pd.read_csv("players(in).csv", index_col=0)
df.head()

,Name,Position,Skill,Salary (€M)
0,Alex Carter,GK,85,90
1,Jordan Smith,GK,88,100
2,Ryan Mitchell,GK,83,85
3,Chris Thompson,GK,80,80
4,Blake Henderson,GK,87,95


* create a list with all records of the players as a dict

In [87]:
players = df.to_dict(orient="records")
players

[{'Name': 'Alex Carter', 'Position': 'GK', 'Skill': 85, 'Salary (€M)': 90},
 {'Name': 'Jordan Smith', 'Position': 'GK', 'Skill': 88, 'Salary (€M)': 100},
 {'Name': 'Ryan Mitchell', 'Position': 'GK', 'Skill': 83, 'Salary (€M)': 85},
 {'Name': 'Chris Thompson', 'Position': 'GK', 'Skill': 80, 'Salary (€M)': 80},
 {'Name': 'Blake Henderson', 'Position': 'GK', 'Skill': 87, 'Salary (€M)': 95},
 {'Name': 'Daniel Foster', 'Position': 'DEF', 'Skill': 90, 'Salary (€M)': 110},
 {'Name': 'Lucas Bennett', 'Position': 'DEF', 'Skill': 85, 'Salary (€M)': 90},
 {'Name': 'Owen Parker', 'Position': 'DEF', 'Skill': 88, 'Salary (€M)': 100},
 {'Name': 'Ethan Howard', 'Position': 'DEF', 'Skill': 80, 'Salary (€M)': 70},
 {'Name': 'Mason Reed', 'Position': 'DEF', 'Skill': 82, 'Salary (€M)': 75},
 {'Name': 'Logan Brooks', 'Position': 'DEF', 'Skill': 86, 'Salary (€M)': 95},
 {'Name': 'Caleb Fisher', 'Position': 'DEF', 'Skill': 84, 'Salary (€M)': 85},
 {'Name': 'Nathan Wright', 'Position': 'MID', 'Skill': 92, 'Sa

<a class="anchor" id="3">

# 3. Problem Definiton
    
</a>

In a fantasy sports league, the objective is to assign players to teams in a way that ensures
a balanced distribution of talent while staying within salary caps.

1) Each player is defined by the following attributes:
* Skill rating: Represents the player's ability.
* Cost: The player's salary.
* Position (One of four roles) : Goalkeeper (GK), Defender (DEF), Midfielder (MID), or Forward (FWD).

A solution is a complete league configuration, specifying the team assignment for each player. These are the constraints that must be verified in every solution of the search space (no object is considered a solution if it doesn’t comply with these):
* Each team must consist of: 1 Goalkeeper, 2 Defenders, 2 Midfielders and 2
Forwards.
* Each player is assigned to exactly one team.

*Impossible Configurations*: Teams that do not follow this exact structure (e.g., a team with 2 goalkeepers, or a team where the same defender is assigned twice) are not part of the search space and are not considered solutions. It is forbidden to generate such an arrangement during evolution.

Besides that, each team should not exceed a 750€ million total budget. If it does, it is not a valid solution and the fitness value should reflect that.

The `objective` is to create a balanced league that complies with the constraints. 
A balanced league a is a league where the average skill rating of the players is roughly the same among the teams. 
This can be measured by the standard deviation of the average skill rating of the teams.

You can find a dataset of players with their names, position, skill rating and salary (in million €).
These players should be distributed across 5 teams of 7 players each.

# 4.Representação com classes player, team and League 

In [88]:
players # list of players, each player is a dictionary

[{'Name': 'Alex Carter', 'Position': 'GK', 'Skill': 85, 'Salary (€M)': 90},
 {'Name': 'Jordan Smith', 'Position': 'GK', 'Skill': 88, 'Salary (€M)': 100},
 {'Name': 'Ryan Mitchell', 'Position': 'GK', 'Skill': 83, 'Salary (€M)': 85},
 {'Name': 'Chris Thompson', 'Position': 'GK', 'Skill': 80, 'Salary (€M)': 80},
 {'Name': 'Blake Henderson', 'Position': 'GK', 'Skill': 87, 'Salary (€M)': 95},
 {'Name': 'Daniel Foster', 'Position': 'DEF', 'Skill': 90, 'Salary (€M)': 110},
 {'Name': 'Lucas Bennett', 'Position': 'DEF', 'Skill': 85, 'Salary (€M)': 90},
 {'Name': 'Owen Parker', 'Position': 'DEF', 'Skill': 88, 'Salary (€M)': 100},
 {'Name': 'Ethan Howard', 'Position': 'DEF', 'Skill': 80, 'Salary (€M)': 70},
 {'Name': 'Mason Reed', 'Position': 'DEF', 'Skill': 82, 'Salary (€M)': 75},
 {'Name': 'Logan Brooks', 'Position': 'DEF', 'Skill': 86, 'Salary (€M)': 95},
 {'Name': 'Caleb Fisher', 'Position': 'DEF', 'Skill': 84, 'Salary (€M)': 85},
 {'Name': 'Nathan Wright', 'Position': 'MID', 'Skill': 92, 'Sa

In [89]:
df

,Name,Position,Skill,Salary (€M)
0,Alex Carter,GK,85,90
1,Jordan Smith,GK,88,100
2,Ryan Mitchell,GK,83,85
3,Chris Thompson,GK,80,80
4,Blake Henderson,GK,87,95
5,Daniel Foster,DEF,90,110
6,Lucas Bennett,DEF,85,90
7,Owen Parker,DEF,88,100
8,Ethan Howard,DEF,80,70
9,Mason Reed,DEF,82,75


In [90]:
# change the column names to lowercase
df.rename(columns={
    "Name": "name",
    "Position": "position",
    "Skill": "skill",
    "Salary (€M)": "salary"
}, inplace=True)

In [91]:
df

,name,position,skill,salary
0,Alex Carter,GK,85,90
1,Jordan Smith,GK,88,100
2,Ryan Mitchell,GK,83,85
3,Chris Thompson,GK,80,80
4,Blake Henderson,GK,87,95
5,Daniel Foster,DEF,90,110
6,Lucas Bennett,DEF,85,90
7,Owen Parker,DEF,88,100
8,Ethan Howard,DEF,80,70
9,Mason Reed,DEF,82,75


In [92]:
position_order = ["GK", "DEF", "MID", "FWD"] # the order of positions

# Create a mapping dictionary: {'GK': 0, 'DEF': 1, 'MID': 2, 'FWD': 3}
# x.map(...) converts each "position" value in the DataFrame to its corresponding order index
# Sorts the DataFrame according to these mapped indices
df_sorted = df.sort_values(by="position", key=lambda x: x.map({pos: i for i, pos in enumerate(position_order)})).reset_index(drop=True)
df_sorted

,name,position,skill,salary
0,Alex Carter,GK,85,90
1,Jordan Smith,GK,88,100
2,Ryan Mitchell,GK,83,85
3,Chris Thompson,GK,80,80
4,Blake Henderson,GK,87,95
5,Daniel Foster,DEF,90,110
6,Lucas Bennett,DEF,85,90
7,Owen Parker,DEF,88,100
8,Ethan Howard,DEF,80,70
9,Mason Reed,DEF,82,75


In [93]:
df_sorted["id"]= df_sorted.index

In [94]:
df_sorted


,name,position,skill,salary,id
0,Alex Carter,GK,85,90,0
1,Jordan Smith,GK,88,100,1
2,Ryan Mitchell,GK,83,85,2
3,Chris Thompson,GK,80,80,3
4,Blake Henderson,GK,87,95,4
5,Daniel Foster,DEF,90,110,5
6,Lucas Bennett,DEF,85,90,6
7,Owen Parker,DEF,88,100,7
8,Ethan Howard,DEF,80,70,8
9,Mason Reed,DEF,82,75,9


In [95]:
players = df_sorted.to_dict(orient="records")
players

[{'name': 'Alex Carter', 'position': 'GK', 'skill': 85, 'salary': 90, 'id': 0},
 {'name': 'Jordan Smith',
  'position': 'GK',
  'skill': 88,
  'salary': 100,
  'id': 1},
 {'name': 'Ryan Mitchell',
  'position': 'GK',
  'skill': 83,
  'salary': 85,
  'id': 2},
 {'name': 'Chris Thompson',
  'position': 'GK',
  'skill': 80,
  'salary': 80,
  'id': 3},
 {'name': 'Blake Henderson',
  'position': 'GK',
  'skill': 87,
  'salary': 95,
  'id': 4},
 {'name': 'Daniel Foster',
  'position': 'DEF',
  'skill': 90,
  'salary': 110,
  'id': 5},
 {'name': 'Lucas Bennett',
  'position': 'DEF',
  'skill': 85,
  'salary': 90,
  'id': 6},
 {'name': 'Owen Parker',
  'position': 'DEF',
  'skill': 88,
  'salary': 100,
  'id': 7},
 {'name': 'Ethan Howard',
  'position': 'DEF',
  'skill': 80,
  'salary': 70,
  'id': 8},
 {'name': 'Mason Reed', 'position': 'DEF', 'skill': 82, 'salary': 75, 'id': 9},
 {'name': 'Logan Brooks',
  'position': 'DEF',
  'skill': 86,
  'salary': 95,
  'id': 10},
 {'name': 'Caleb Fisher

In [96]:
team_order= ["GK", "DEF", "MID", "FWD"]
team_grouped = df_sorted.groupby("position")["id"].apply(list)
position_id_map = {pos: team_grouped[pos] for pos in team_order if pos in team_grouped}

In [97]:
position_id_map

{'GK': [0, 1, 2, 3, 4],
 'DEF': [5, 6, 7, 8, 9, 10, 11, 12, 13, 14],
 'MID': [15, 16, 17, 18, 19, 20, 21, 22, 23, 24],
 'FWD': [25, 26, 27, 28, 29, 30, 31, 32, 33, 34]}

In [98]:
import random


In [99]:
def generate_team_indices(players,position_id_map):
    # generates a team from a dictionary of indices

    # Randomly select one valid team (just player indices)
    team = []
    team += random.sample(position_id_map["GK"], 1)  # Select 1 goalkeeper
    team += random.sample(position_id_map["DEF"], 2) # Select 2 defenders
    team += random.sample(position_id_map["MID"], 2) # Select 2 midfielders
    team += random.sample(position_id_map["FWD"], 2) # Select 2 forwards

    return team

In [100]:
team_indices = generate_team_indices(players,position_id_map)
team_indices

[1, 9, 6, 22, 18, 30, 34]

In [101]:
team_indices = generate_team_indices(players,position_id_map)  

print("Team (indices):", team_indices)
print("Team (players):")
for i in team_indices:
    p = players[i]
    print(f"- {p['name']} ({p['position']}) Skill: {p['skill']}, Salary: €{p['salary']}M")

Team (indices): [3, 11, 14, 16, 23, 28, 34]
Team (players):
- Chris Thompson (GK) Skill: 80, Salary: €80M
- Caleb Fisher (DEF) Skill: 84, Salary: €85M
- Brayden Hughes (DEF) Skill: 87, Salary: €100M
- Hunter Cooper (MID) Skill: 83, Salary: €85M
- Ashton Phillips (MID) Skill: 90, Salary: €110M
- Sebastian Perry (FWD) Skill: 95, Salary: €150M
- Zachary Nelson (FWD) Skill: 86, Salary: €92M


In [102]:
def generate_league(players):
    from copy import deepcopy
    import random
    num_teams=5
    
    # Step 1: Copy the player pool
    available_indices = list(range(len(players)))
    random.shuffle(available_indices)  # randomize pool to start
    
    # Step 2: Group indices by position
    def group_available(indices):
        from collections import defaultdict
        pos_map = defaultdict(list)
        for i in indices:
            pos_map[players[i]["position"]].append(i)
        return pos_map
    
    league = []

    for _ in range(num_teams):
        pos_to_indices = group_available(available_indices)

        # Check we still have enough players per role
        if (len(pos_to_indices["GK"]) < 1 or
            len(pos_to_indices["DEF"]) < 2 or
            len(pos_to_indices["MID"]) < 2 or
            len(pos_to_indices["FWD"]) < 2):
            raise ValueError("Not enough players left to form a complete team.")

        team = []
        team += random.sample(pos_to_indices["GK"], 1)
        team += random.sample(pos_to_indices["DEF"], 2)
        team += random.sample(pos_to_indices["MID"], 2)
        team += random.sample(pos_to_indices["FWD"], 2)

        # Remove these players from the available pool
        for idx in team:
            available_indices.remove(idx)

        league.append(team)

    return league


In [106]:
league = generate_league(players)  # this gives you a league

# Just display the structure (no formatting, no printing names)
league

[[4, 10, 12, 20, 18, 27, 34],
 [3, 13, 7, 24, 15, 33, 32],
 [1, 14, 5, 22, 19, 28, 26],
 [0, 8, 9, 23, 16, 31, 30],
 [2, 11, 6, 17, 21, 25, 29]]

# SOLUTION REPRESENTATION

In [107]:
from abc import ABC, abstractmethod

In [108]:
class Solution(ABC):
    def __init__(self, repr=None):
        # To initialize a solution we need to know it's representation.
        # If no representation is given, a representation is randomly initialized.
        if repr == None:
            repr = self.random_initial_representation()
        # Attributes
        self.repr = repr

    # Method that is called when we run print(object of the class)
    def __repr__(self):
        return str(self.repr)

    # Other methods that must be implemented in subclasses
    @abstractmethod
    def fitness(self):
        pass

    @abstractmethod
    def random_initial_representation():
        pass


In [109]:
df_sorted.head(5)

,name,position,skill,salary,id
0,Alex Carter,GK,85,90,0
1,Jordan Smith,GK,88,100,1
2,Ryan Mitchell,GK,83,85,2
3,Chris Thompson,GK,80,80,3
4,Blake Henderson,GK,87,95,4


In [110]:
def generate_league(df):

    import random
    from collections import defaultdict
    
    num_teams=5
    available_ids = df.index.tolist()
    random.shuffle(available_ids)

    league = []

    for _ in range(num_teams):
        pos_map = defaultdict(list)

        # Build position map using current available players
        for i in available_ids:
            pos = df.loc[i, "position"]
            pos_map[pos].append(i)

        # Check we have enough players left per role
        if (len(pos_map["GK"]) < 1 or
            len(pos_map["DEF"]) < 2 or
            len(pos_map["MID"]) < 2 or
            len(pos_map["FWD"]) < 2):
            raise ValueError("Not enough players left to form a full team")

        # Select players for the team
        team = []
        team += random.sample(pos_map["GK"], 1)
        team += random.sample(pos_map["DEF"], 2)
        team += random.sample(pos_map["MID"], 2)
        team += random.sample(pos_map["FWD"], 2)

        # Remove them from pool
        for idx in team:
            available_ids.remove(idx)

        league.append(team)

    return league

In [113]:
class SportsLeagueSolution(Solution):
    def __init__(self, repr=None, players_df=df_sorted):
        self.players_df = players_df
        super().__init__(repr=repr)

    def random_initial_representation(self):
        self.repr = generate_league(self.players_df)
        return self.repr

    def fitness(self):
        current_league = self.repr  # list of team index lists
        team_skills = []
        team_salaries = []

        for current_league in league:
            team_df = self.players_df.loc[team]
            total_skill = team_df["skill"].sum()
            total_salary = team_df["salary"].sum()

            # Budget constraint
            if total_salary > 750:
                return 1e9  

            team_skills.append(total_skill)
            team_salaries.append(total_salary)

        avg_skill = sum(team_skills) / len(team_skills)
        salary_std = pd.Series(team_salaries).std()

        # Maximize skill, penalize unbalanced salary
        return avg_skill - 0.1 * salary_std


In [114]:
sol1 = SportsLeagueSolution()
sol1

[[0, 12, 9, 22, 20, 31, 25], [3, 6, 10, 15, 16, 33, 34], [2, 8, 14, 18, 17, 30, 26], [1, 11, 7, 19, 23, 28, 27], [4, 5, 13, 24, 21, 29, 32]]

In [115]:
sol2 = SportsLeagueSolution()
sol2

[[3, 12, 7, 23, 18, 25, 28], [4, 6, 13, 24, 22, 27, 30], [0, 5, 11, 19, 15, 29, 31], [1, 10, 9, 16, 21, 26, 34], [2, 8, 14, 17, 20, 33, 32]]

In [82]:
position_id_map

{'GK': [0, 1, 2, 3, 4],
 'DEF': [5, 6, 7, 8, 9, 10, 11, 12, 13, 14],
 'MID': [15, 16, 17, 18, 19, 20, 21, 22, 23, 24],
 'FWD': [25, 26, 27, 28, 29, 30, 31, 32, 33, 34]}

* sugestão de crossover (exemplo): podem, por exemplo, manter todos os jogadores de uma determinada posição do pai1 no filho1, 
nas mesmas equipas em que se encontram no pai1. Depois, para outra posição, podem usar a distribuição desses jogadores pelas equipas do pai2, e assim sucessivamente.

* A vossa sugestão de “trocar jogadores entre equipas dentro da mesma liga” é considerado uma mutação e não um crossover.